<a href="https://colab.research.google.com/github/ViekyKayal/jobQA_chatbot/blob/main/jobsQA__chatbot(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!pip install -r /content/drive/MyDrive/chatbot2/req.txt

In [ ]:
!pip install --upgrade --quiet docx2txt

  Preparing metadata (setup.py) ... done


In [ ]:
import os
import io
import PyPDF2
from google.colab import files
from langchain import PromptTemplate
from langchain_community.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema import StrOutputParser


In [ ]:
#upload your CV here
uploaded = files.upload()

pdf_file = list(uploaded.keys())[0]
pdf_reader = PyPDF2.PdfReader(io.BytesIO(uploaded[pdf_file]))

text=""
for page in pdf_reader.pages:
  text += page.extract_text()
print(text)

Saving cv.pdf to cv.pdf
Curriculum vitae  
 
Full Name  : Vieky Kayal  
Contact  : 9674309252  
Email   : viekykayal19@gmail.c om 
Address  : 18/1a Dr A K  Paul Road, Behala Chowrasta, Kol -34 
 
 
OBJECTIVE :  
To harness my abilities fully in contributing to the growth of the organization and ensuring self -progress through hard -
work and determination along with continuously upgrading my knowledge and skills.  
 
ACADEMIC CREDENTIALS:  
Degree  
  INSTITUTION 
NAME  
  BOARD  
 PERCENTAGE/ 
CGPA  
 YEAR OF 
PASSING  
 
1. B. Tech 
in IT   St. Thomas’ 
College of 
Engineering & 
Technology  
 MAULANA 
ABUL KALAM 
AZAD 
UNIVERSITY OF 
TECHNOLOGY  
 1st sem 9.69 
2nd sem 9.39 
3rd sem 9.64 
4th sem 9.67 
5th sem 8.63 
 2024  
2. Class XII  Army Public School 
Ballygunge  CBSE  87.6% 2019  
3. Class X  Army Public School 
Jammu Cantt.  CBSE  A1( 95% )  2017  
 
TECHNICAL LANGUAGE PROFICIENCY : C, C++, Python  
CO-CURRICULAR ACTIVITIES:  
1. Received NPTEL ELITE+GOLD in Problem Solving 

In [ ]:
DB_FAISS_PATH ="/content/drive/MyDrive/chatbot2/jobs"
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
custom_prompt_template = """ user data is present in USER_DATA: {user}
    available jobs are present in JOBS_DATA: {context}

You are a job agent and you do the any of following work:

1. remember user details present in USER_DATA
2. find jobs present in JOBS_DATA based on user skills present in USER_DATA
3. answer user question based on discussion of the current job

 Use the above pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Question: {question}

Only return the result below  and nothing else.
<$#ANSWER RESULT#$>:"""

In [ ]:

def set_custom_prompt():
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'],
                           partial_variables={"user": text},)
    return prompt



In [ ]:
def load_llm():
    # Load the locally downloaded model here

    return HuggingFaceHub(  repo_id="HuggingFaceH4/zephyr-7b-beta",
                            model_kwargs={"temperature": 0.1, "max_length": 64,"max_new_tokens":512}
)

In [ ]:
def load_llm():
    # Load the locally downloaded model here

    return HuggingFaceHub(  repo_id="huggingfaceh4/zephyr-7b-alpha",
                            model_kwargs={"temperature": 0.1, "max_length": 64,"max_new_tokens":512}
)

In [ ]:
#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                            chain_type='stuff',
                                            retriever = db.as_retriever(search_kwargs={"k": 5}),
                                            return_source_documents = False,
                                            chain_type_kwargs={'prompt': prompt,
                                                                "memory": ConversationBufferMemory( memory_key="history",
                                                                                                    input_key="question")},
                                          )
    return qa_chain

In [ ]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db= FAISS.load_local(DB_FAISS_PATH, embeddings,allow_dangerous_deserialization=True)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [ ]:
#output function

def chatbot():
  exit=['bye','tata', 'exit']
  exit_text='Bye, Have a nice day'
  qa_result = qa_bot()
  while True:
    query=input("user: ")
    if query in exit: return "chatbot:\n "+exit_text
    answer=qa_result({'query': query})
    print("chatbot: "+answer['result'].split('<$#ANSWER RESULT#$>:')[-1])

In [ ]:
print(chatbot())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


user: what is my name?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


chatbot: 
Vieky Kayal
user: what is my highest qualification?
chatbot: 
Degree: B. Tech in IT from St. Thomas’ College of Engineering & Technology, recognized by MAULANA ABUL KALAM AZAD UNIVERSITY OF TECHNOLOGY, with a CGPA of 9.67 in the final semester. Passed out in 2024.
user: tell me about the available java jobs
chatbot: 
We have identified a Java Developer position at Delphic with a minimum of 5 years of industry experience required. The job is full-time and remote, with a company size of 51-200 employees and a focus on software development. The ideal candidate will have a strong background in Java development and will be responsible for designing, developing, and maintaining Java-based applications and microservices. They will also be expected to stay updated on emerging technologies and best practices in Java development and to mentor junior developers. If you meet these requirements, we encourage you to apply for this opportunity.
user: what are requirements for this jobs?
cha

['Bye, Have a nice day']

vector store

In [ ]:
#creating a vector store
DataBase_PATH ="/content/drive/MyDrive/chatbot2/jobs/"
def create_vs(DB_PATH):
  loader_classes= Docx2txtLoader
  loader = DirectoryLoader(DB_PATH, glob='*.docx', loader_cls= loader_classes )
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  db = FAISS.from_documents(texts, embeddings)
  db.save_local(DB_FAISS_PATH)
  retriever_vectordb = db.as_retriever(search_kwargs={"k": 10})
  return retriever_vectordb
retriever_vectordb = create_vs(DataBase_PATH)

In [ ]:
import pandas as pd

def show_vStore(store):
  vector_df= store_to_df(store)
  display(vector_df)

def store_to_df(store):
  v_dict= store.docstore._dict
  data_rows=[]
  for k in v_dict.keys():
    doc_name= v_dict[k].metadata['source'].split('/')[-1]
    #page_number= v_dict[k].metadata['page']+1
    content= v_dict[k].page_content
    data_rows.append({"chunk_id":k, "document": doc_name, "content": content})
  vector_df=pd.DataFrame(data_rows)
  return vector_df

#show_vStore(new_db)

In [ ]:
def add_to_vector_store(directory, store):
  if directory[-5:] == '.docx':
    loader = Docx2txtLoader(directory)
  else:
    loader= DirectoryLoader(directory, loader_cls=Docx2txtLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  extension= FAISS.from_documents(texts, embeddings)
  store.merge_from(extension)

def delete_document(store,document):
  vector_df= store_to_df(store)
  if document not in vector_df['document']:
    print("no file found")
    return
  chunks_list = vector_df.loc[vector_df['document']==document]['chunk_id'].tolist()
  store.delete(chunks_list)




previous outputs

In [ ]:
chatbot()


chatbot: 
Vieky Kayal
chatbot: 

Job Description for Web Developer at https://aladdin365.com/:

Join the ultimate textile and apparel networking platform 	Where every voice is heard	Every idea is valued	Every dream becomes a reality

At https://aladdin365.com/, we are looking for a talented and experienced Web Developer to join our team. The successful candidate will be responsible for developing and maintaining our website, as well as working on various web-based projects.

Key Responsibilities:

- Develop and maintain our website using HTML, CSS, JavaScript, and PHP
- Collaborate with the design team to ensure that the website is visually appealing and user-friendly
- Implement new features and functionality as required
- Optimize the website for performance and search engine optimization (SEO)
- Troubleshoot and resolve any issues that arise with the website
- Stay up-to-date with the latest web development technologies and trends

Requirements:

- Bachelor's degree in Computer Scie

In [ ]:

print("chatbot:",cb['result'])

chatbot:  user data is present in USER_DATA: Curriculum vitae  
 
Full Name  : Vieky Kayal  
Contact  : 9674309252  
Email   : viekykayal19@gmail.c om 
Address  : 18/1A  Dr A K  Paul Road, Behala Chowrasta, Kol -34 
 
OBJECTIVE :  
To harness my abilities fully in contributing to the growth of the organization and ensuring self -progress through hard -
work and determination along with continuously upgrading my knowledge and skills.  
 
ACADEMIC CREDENTIALS:  
DEGREE  INSTITUTION NAME  BOARD  PERCENTAGE/ 
CGPA  YEAR OF 
PASSING  
1. B. Tech in 
Information 
Technology   St. Thomas’ College of 
Engineering & Technology  Maul ana Abul Kalam 
Azad University o f 
Technology  1st sem 9.69 
 
2024  2nd sem 9.39 
3rd sem 9.64 
4th sem 9.67 
5th sem 8.63 
6th sem 9.32 
2. Class XII  Army Public School, 
Ballygunge  CBSE  87.6%  2019  
3. Class X  Army Public School , Jammu 
Cantt.  CBSE  A1( 95% )  2017  
TECHNICAL LANGUAGE PROFICIENCY :  
1. Language known: C, C++, Python , Java , oracle SQL

In [ ]:
query="who is arka"
cb=final_result(query)
print(cb)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'who is arka', 'result': 'Arka is a young boy in Willowbrook who is known for his curiosity and love of adventure. He is inspired by the tales of far-off lands, hidden treasures, and mythical creatures, and he spends his days reading about daring explorers and their incredible adventures.', 'source_documents': [Document(page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed.", metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}), Document(page_content='Title: "Arka\'s Daring Adventure"  \nOnce upon a time in the quaint village of Willowbrook, there lived a curious and adventurous young \nboy named Arka. Arka had always been fascinated by tales of far -off lands

In [ ]:
print("user:",cb['query'])
print("chatbot:",cb['result'])

user: who is arka
chatbot: Arka is a young boy in Willowbrook who is known for his curiosity and love of adventure. He is inspired by the tales of far-off lands, hidden treasures, and mythical creatures, and he spends his days reading about daring explorers and their incredible adventures.


In [ ]:
while True:
  query=input("user:")
  if query=="exit": break
  cb=final_result(query)
  print("chatbot:",cb['result'])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chatbot: Arka has friends in Willowbrook, as he mentioned that he would visit them whenever he could.
user:name of his friend


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chatbot: Arka's friend's name is Polly.


KeyboardInterrupt: ignored

In [ ]:
for i in cb:
  print(i,":",cb[i])
for i in cb["source_documents"]:
  print(i)

query : who is his friend
result : Arka's friend is Polly.
source_documents : [Document(page_content='made his final wish to return ho me, promising to visit the Isle of Enchantment whenever he could.  \nWith a heavy heart, he and Polly bid farewell to the magical island, but they knew that their \nadventures would continue in Willowbrook and beyond.  \nArka returned to his village, where he shared his incredible journey with everyone. The lessons he \nhad learned and the courage he had gained from the Isle of Enchantment made him a beloved figure', metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}), Document(page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed.", me

In [ ]:
for i in cb["source_documents"]:
  print(str(i))

page_content='made his final wish to return ho me, promising to visit the Isle of Enchantment whenever he could.  \nWith a heavy heart, he and Polly bid farewell to the magical island, but they knew that their \nadventures would continue in Willowbrook and beyond.  \nArka returned to his village, where he shared his incredible journey with everyone. The lessons he \nhad learned and the courage he had gained from the Isle of Enchantment made him a beloved figure' metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}
page_content="in Willowbrook, known far and wide as the adventurous boy wh o had truly lived his dreams.  \nAnd so, Arka's story became a legend in its own right, inspiring generations of children to dream big \nand embark on their own daring adventures, for they knew that the world was full of magic and \nwonder, waiting to be discover ed." metadata={'source': '/content/drive/MyDrive/chatbot/llama/data/story.pdf', 'page': 0}


In [ ]:
https://python.langchain.com/docs/integrations/memory/sql_chat_message_history

In [ ]:
https://python.langchain.com/docs/integrations/retrievers/self_query/timescalevector_self_query

In [ ]:
# Install required libraries
!pip install transformers langchain pymongo google-colab

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["your_database_name"]

# Load Llama 2 model
llm = AutoModelForCausalLM.from_pretrained("facebook/llama-2-7b")
tokenizer = AutoTokenizer.from_pretrained("facebook/llama-2-7b")

# Define functions for retrieving data from MongoDB
def get_document(query):
  # Implement logic to search documents in MongoDB based on the query
  # and return relevant information
  pass

def get_similar_documents(query, doc_id):
  # Implement logic to find similar documents from MongoDB based on
  # the query and provided document ID
  pass


In [ ]:
from langchain.chains import ConversationalRetrievalChain

# Define text splitter and embedding extractor
text_splitter = RecursiveCharacterTextSplitter()
embeddings = HuggingFaceEmbeddings(llm_model=llm)

# Connect LLM, data retrieval functions, and memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retrieval_function=lambda query: (get_document(query), None),
    similar_documents_function=lambda query, doc_id: get_similar_documents(query, doc_id),
    text_splitter=text_splitter,
    embeddings=embeddings,
    return_source_documents=True,
)

# Define chat history memory
memory = ConversationBufferMemory(maxlen=10)

# Combine all parts into a single chain
chatbot_chain = qa_chain.with_memory(memory)


In [ ]:
while True:
  # Get user query
  query = input("Ask me anything: ")

  # Run the chatbot chain with the query and chat history
  result = chatbot_chain({"question": query, "chat_history": memory.get_all()})

  # Extract and display the answer
  answer = result["answer"]
  print(f"Chatbot: {answer}")

  # Update chat history with current conversation
  memory.update({"question": query, "answer": answer})
